안녕하세요! 

데이크루 1기로 활동 중인 sssssun입니다!

계속해서 sklearn을 활용하여 머신러닝 모델의 성능을 높이는 법을 공부하고 있는데요.

이번 포스팅은 "**앙상블 학습(Ensemble Learning)**"에 대해 다루어보려고 합니다.




### **앙상블 학습(Ensemble Learning)**

앙상블 학습(Ensemble Learning)은 여러 개의 분류기를 생성하고, 그 예측을 결합함으로써 보다 정확한 예측을 도출하는 기법을 말합니다.

회귀 모델이든 분류 모델이든, 하나의 모델을 사용하기 보다는 모델 여러 개를 조합하여 더 정확한 예측에 도움을 주는 방식입니다.

구체적으로 여러 개의 약 분류기 (Weak Classifier)를 결합하여 강 분류기(Strong Classifier)를 만들어 모델의 정확성을 향상시킵니다.

예를 들어 보면, 어떤 문제를 해결하는 데 있어서 한 명의 전문가보다는 여러 명의 비전문가 일반인이 집단지성을 이용하여 문제를 해결하는 방식이 나을 수도 있습니다.

이게 바로 앙상블 기법이라고 할 수 있죠.

실제로 머신러닝 대회에서 우승하는 솔루션들은 대부분 앙상블 기법을 활용해 고성능을 낸다고 합니다.

### **앙상블 학습 유형**

앙상블 학습은 크게 보팅(Voting), 배깅(Bagging), 부스팅(Boosting)으로 나눌 수 있습니다.



### **1. 보팅(Voting)**

말 그대로 여러 개의 분류기가 투표를 통해 최종 예측 결과를 결정하는 방식입니다.

서로 다른 알고리즘을 여러 개 결합하여 사용하게 됩니다.

- **하드 보팅(Hard Voting)**

: 다수의 분류기가 예측한 결과값을 최종 결과로 선정, 즉 다수결의 원칙을 따릅니다.

- **소프트 보팅(Soft Voting)**

: 모든 분류기가 예측한 레이블 값의 결정 확률 평균을 구한 뒤 가장 확률이 높은 레이블 값을 최종 결과로 선정합니다.

sklearn의 유방암 데이터셋을 활용하여 보팅 방식의 앙상블 학습을 구현해보겠습니다.

라이브러리를 import 해주었고, 데이터셋을 불러왔습니다.

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

cancer = load_breast_cancer()

In [5]:
cancer = pd.DataFrame(cancer.data, columns=cancer.feature_names)
cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


그리고 데이터셋까지 훈련 데이터셋과 테스트 데이터셋으로 나눠주었습니다.

In [10]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2 , random_state= 156)

sklearn의 보팅 분류기, **VotingClassifier** 를 활용해보겠습니다.

그리고 보팅 분류기에 넣을 개별 모델은 **로지스틱 회귀 모델**과 **KNN 모델**을 선택해주었습니다.

In [14]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

로지스틱 회귀와 KNN을 각각 객체 지정해주고, 둘다 VotingClassifier에 넣어줍니다.

voting 매개변수를 통해 타입을 선택해줄 수 있는데, 저는 **soft voting** 으로 해주었습니다.

In [23]:
lr = LogisticRegression()
knn = KNeighborsClassifier(n_neighbors=8)

V_clf = VotingClassifier( estimators=[('LR',lr),('KNN',knn)] , voting='soft' )

그리고 다른 알고리즘을 학습하는 것처럼, VotingClassifier 또한 학습, 예측 그리고 평가의 과정을 거쳐줍니다.

In [24]:
V_clf.fit(X_train , y_train)
pred = V_clf.predict(X_test)
print('VotingClassifier 정확도: ', round(accuracy_score(y_test , pred),4))

VotingClassifier 정확도:  0.9474


로지스틱 회귀 모델, KNN 모델 각각 한가지만 활용했을 때 정확도를 구해보겠습니다.

In [25]:
models = [lr, knn]
for model in models:
    model.fit(X_train , y_train)
    pred = model.predict(X_test)
    name = model.__class__.__name__
    print('{0} 정확도: {1:.4f}'.format(name, accuracy_score(y_test , pred)))

LogisticRegression 정확도: 0.9386
KNeighborsClassifier 정확도: 0.9386


즉, VotingClassifier를 사용하였을 때는 정확도가 0.9474, 나머지 모델들을 한 개씩만 이용하였을 때는 정확도가 0.9386이 나오네요.

보팅 분류기를 이용하는 것이 더 정확도가 높은 것을 볼 수 있었습니다.

### **2. 배깅(Bagging)**

배깅은 간단히 말해 **훈련 세트의 중복을 허용하여 샘플링**을 하는 방식입니다. 

통계학에서는 "부트스트래핑"이라고 부르는데,

각 모델이 서로 다른 학습데이터셋을 이용해야하고, 데이터 셋을 뽑을때는 복원추출을 한다는 점, 그리고 원 데이터 수만큼 데이터 셋을 뽑아야 합니다.

배깅은 다음과 같은 과정을 거칩니다!

1. 동일한 알고리즘을 사용하는 일정 수의 분류기를 생성합니다.

2. 각각의 분류기에 부트스트래핑 분할 방식으로 생성된 샘플데이터를 학습합니다.

3. 최종적으로 각각의 결과를 보팅(Voting)을 통해 예측 결정합니다.


### **랜덤 포레스트(Random Forest)**

배깅의 대표적인 알고리즘은 **랜덤 포레스트**입니다.

랜덤 포레스트는 여러 개의 결정 트리 분류기가 전체 데이터에서 배깅 방식으로 각자의 데이터를 샘플링합니다.

그리고 개별적으로 학습을 수행하고, 최종적으로 모든 분류기가 보팅을 통해 예측 결정을 합니다.

In [26]:
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, test_size=0.2 , random_state= 156)

RandomForestClassifier를 import 해주었습니다.

In [29]:
from sklearn.ensemble import RandomForestClassifier

사용법은 하나의 알고리즘과 동일하게 객체 지정, 학습, 예측의 과정을 거쳐주면 됩니다.

In [33]:
clf = RandomForestClassifier(n_estimators=300, max_depth=20,random_state=0)
clf.fit(X_train,y_train)
pred1 = clf.predict(X_test)

In [32]:
print('RandomForestClassifier 정확도: ', round(accuracy_score(y_test,pred1),4))

RandomForestClassifier 정확도:  0.9561


정확도가 0.9561 이네요. 보팅 분류기보다 높습니다.

랜덤 포레스트는 다양한 하이퍼 파라미터를 갖기 때문에, 하이퍼 파라미터 튜닝을 해주면 더 좋은데요.

마침 지난 포스팅에서 다뤘던 GridSearchCV를 이용한 하이퍼 파라미터 튜닝을 해보겠습니다.


In [42]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100],
    'max_depth' : [6, 8, 10, 12], 
    'min_samples_leaf' : [8, 12, 18 ],
    'min_samples_split' : [8, 16, 20]
}


rf_clf = RandomForestClassifier(random_state=0, n_jobs=-1)
grid_cv = GridSearchCV(rf_clf , param_grid=params , cv=2, n_jobs=-1 )
grid_cv.fit(X_train , y_train)

print('최적 하이퍼 파라미터:\n', grid_cv.best_params_)
print('\n최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:
 {'max_depth': 6, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 100}

최고 예측 정확도: 0.9451


### **3. 부스팅(Boosting)**


부스팅은 여러 개의 약한 학습기(weak learner)를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치를 부여해 오류를 개선해나가며 학습하는 방식입니다.

부스팅(Boosting)은 편향(bias)이 높은 경우 이를 낮추는 것과 같이 성능 자체를 강화하는데 목적을 둡니다. 



부스팅 알고리즘은 대표적으로 다음과 같은 알고리즘이 있습니다.

- AdaBoost
- Gradient Booting Machine(GBM)
- XGBoost
- LightGBM
- CatBoost

다른 분들의 코드에서 많이 보았던 알고리즘들인 것 같아요!

사용법은 랜덤 포레스트 및 다른 알고리즘들과 동일합니다.

AdaBoostClassifier와 GradientBoostingClassifier를 구현해보겠습니다.

In [38]:
from sklearn.ensemble import AdaBoostClassifier


clf = AdaBoostClassifier(n_estimators=30, random_state=10, learning_rate=0.1)
clf.fit(X_train, y_train)
pred2 = clf.predict(X_test)

print('AdaBoost 정확도: ', round(accuracy_score(y_test,pred2),4))

AdaBoost 정확도:  0.9474


In [39]:
from sklearn.ensemble import GradientBoostingClassifier 


gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
pred3 = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, pred3)

print('GBM 정확도: ', round(accuracy_score(y_test,pred3),4))

GBM 정확도:  0.9561


GBM과 랜덤포레스트를 이용했을 때 정확도가 0.9561로 가장 높습니다.

데이터에 따라서 앙상블 방법을 여러가지 시도해보고 선택하면 좋을 것 같아요.

읽어주셔서 감사합니다:)